# CALIDAD DE DATOS

## 1. IMPORTAMOS PAQUETES Y DATOS

In [1]:
import pandas as pd
import zipfile
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
%config IPCompleter.greedy = True
import warnings
warnings.filterwarnings('ignore')

Definimos la ruta donde ubicamos los datasets para no tener que estar poniéndola en cada ocasión

In [2]:
zip_path = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/EVOLVE/Data Science/EVOLVE/Fernando_Costa/Practicas/Mini_Proyecto_EDA/001_crime_from_2020/000_datasets/Crime_Data_from_2020_to_Present.zip'

Definimos el dataset que vamos a usar.

A continuación, aplicaremos los cambios de manera individual sobre el df pero, después, generaremos una función que aglutine todos los pasos individuales y la aplicaremos directamente

In [3]:
csv_filename = 'Crime_Data_from_2020_to_Present.csv'

Cargamos el dataset df

In [4]:
frac = 0.33
chunksize = 100_000  # ajusta según memoria

rows = []

with zipfile.ZipFile(zip_path) as z:
    with z.open(csv_filename) as f:
        for chunk in pd.read_csv(f, chunksize=chunksize):
            rows.append(chunk.sample(frac=frac))
            
df = pd.concat(rows, ignore_index=True)
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,201818775,10/13/2020 12:00:00 AM,10/13/2020 12:00:00 AM,1650,18,Southeast,1829,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,AO,Adult Other,740.0,NaN,NaN,NaN,2200 CENTURY BL,NaN,33.9465,-118.2332
1,201421059,11/23/2020 12:00:00 AM,11/22/2020 12:00:00 AM,1500,14,Pacific,1425,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,3200 SAWTELLE BL,NaN,34.0234,-118.4292
2,202113306,08/31/2020 12:00:00 AM,08/31/2020 12:00:00 AM,1010,21,Topanga,2157,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,20500 HAYNES ST,NaN,34.1895,-118.5811
3,201915505,10/10/2020 12:00:00 AM,10/10/2020 12:00:00 AM,1040,19,Mission,1998,1,442,SHOPLIFTING - PETTY THEFT ($950 & UNDER),...,IC,Invest Cont,442.0,NaN,NaN,NaN,7800 VAN NUYS BL,NaN,0.0000,0.0000
4,201317760,09/30/2020 12:00:00 AM,09/29/2020 12:00:00 AM,1041,13,Newton,1321,1,440,THEFT PLAIN - PETTY ($950 & UNDER),...,IC,Invest Cont,440.0,NaN,NaN,NaN,300 W ADAMS BL,NaN,34.0289,-118.2686


## 2. COMPROBAMOS LAS CARACTERÍSTICAS DEL DATASET

In [5]:
df.shape

(331647, 28)

Comprobamos que el dataset tiene nulos y nos sirve para trabajar

In [6]:
df.isna().sum().sort_values(ascending=False)

Crm Cd 4          331622
Crm Cd 3          330920
Crm Cd 2          308866
Cross Street      280683
Weapon Used Cd    223748
Weapon Desc       223748
Mocodes            50229
Vict Descent       48001
Vict Sex           47997
Premis Desc          183
Premis Cd              5
Crm Cd 1               4
Status                 1
Vict Age               0
Crm Cd                 0
Crm Cd Desc            0
AREA NAME              0
Rpt Dist No            0
AREA                   0
TIME OCC               0
Date Rptd              0
DATE OCC               0
DR_NO                  0
Part 1-2               0
Status Desc            0
LOCATION               0
LAT                    0
LON                    0
dtype: int64

Comprobamos las dimensiones de la tabla y los valores únicos de DR_NO ya que sospecho que debe ser el índice

In [7]:
dimensiones_df = df.shape
print(f'Dimensiones df: ', dimensiones_df)

Dimensiones df:  (331647, 28)


In [8]:
DR_NO = df.DR_NO.nunique()
print(f'Valores únicos de DR_NO: ', DR_NO)

Valores únicos de DR_NO:  331647


CAMBIOS EN NOMBRES E ÍNDICE: 
- Transformamos a minúsculas los nombres de las columnas
- Ponemos 'dr_no' como index ya que es el identificador del reporte 

In [9]:
df.columns = df.columns.str.replace(' ','_').str.lower()
df.columns = df.columns.str.replace('-','_')
df = df.set_index('dr_no')
df

,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,mocodes,...,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
dr_no,,,,,,,,,,,,,,,,,,,,,
201818775,10/13/2020 12:00:00 AM,10/13/2020 12:00:00 AM,1650,18,Southeast,1829,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329 2000 1300 1813 1814,...,AO,Adult Other,740.0,NaN,NaN,NaN,2200 CENTURY BL,NaN,33.9465,-118.2332
201421059,11/23/2020 12:00:00 AM,11/22/2020 12:00:00 AM,1500,14,Pacific,1425,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329 1300 1609,...,IC,Invest Cont,740.0,NaN,NaN,NaN,3200 SAWTELLE BL,NaN,34.0234,-118.4292
202113306,08/31/2020 12:00:00 AM,08/31/2020 12:00:00 AM,1010,21,Topanga,2157,2,624,BATTERY - SIMPLE ASSAULT,0416,...,AO,Adult Other,624.0,NaN,NaN,NaN,20500 HAYNES ST,NaN,34.1895,-118.5811
201915505,10/10/2020 12:00:00 AM,10/10/2020 12:00:00 AM,1040,19,Mission,1998,1,442,SHOPLIFTING - PETTY THEFT ($950 & UNDER),0325,...,IC,Invest Cont,442.0,NaN,NaN,NaN,7800 VAN NUYS BL,NaN,0.0000,0.0000
201317760,09/30/2020 12:00:00 AM,09/29/2020 12:00:00 AM,1041,13,Newton,1321,1,440,THEFT PLAIN - PETTY ($950 & UNDER),0344 1601 0202,...,IC,Invest Cont,440.0,NaN,NaN,NaN,300 W ADAMS BL,NaN,34.0289,-118.2686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240218299,12/28/2024 12:00:00 AM,12/22/2024 12:00:00 AM,1900,2,Rampart,279,1,330,BURGLARY FROM VEHICLE,1300 1609 1307 1822 0344,...,IC,Invest Cont,330.0,NaN,NaN,NaN,1300 W OLYMPIC BL,NaN,34.0473,-118.2701
240608549,04/17/2024 12:00:00 AM,04/17/2024 12:00:00 AM,1130,6,Hollywood,644,1,440,THEFT PLAIN - PETTY ($950 & UNDER),1822 0344,...,IC,Invest Cont,440.0,NaN,NaN,NaN,7100 W SUNSET BL,NaN,34.0980,-118.3441
241210889,04/28/2024 12:00:00 AM,04/28/2024 12:00:00 AM,1230,12,77th Street,1259,2,624,BATTERY - SIMPLE ASSAULT,1309 0416 0448 0419 0913,...,IC,Invest Cont,624.0,NaN,NaN,NaN,300 E FLORENCE AV,NaN,33.9747,-118.2696


## 3. SIGNIFICADO Y FORMATO DE LAS COLUMNAS

Ya sabemos que podemos trabajar con el dataset y lo tenemos cargado con los nombre de las columnas en su formato correto

Significado de las columnas:
- dr_no: Número de expediente oficial compuesto por un año de 2 dígitos, un ID de área y 5 dígitos.
- date_rptd: Fecha de Reporte. Indica el día en que el crimen o incidente fue oficialmente reportado (MM/DD/AAAA)
- date_occ: Fecha de Ocurrencia. Indica el día real en que el crimen tuvo lugar (MM/DD/AAAA)
- time_occ: En horario militar de 24 horas.
- area: Áreas Geográficas o Divisiones de Patrulla numeradas secuencialmente del 1 al 21.
- area_name: Nombre de las áreas Geográficas o Divisiones de Patrulla
- rpt_dist_no: Número de distrito de la patrulla del oficial que informó el incidente.
- part_1_2: Indica si el incidente es un crimen de la Parte 1 (crímenes más graves) o de la Parte 2 (crímenes menos graves).
- crm_cd: Código de delito de 3 dígitos del crimen cometido.
- crm_cd_desc: Descripción del delito del crimen cometido.
- mocodes: Modus Operandi o la manera distintiva o característica en que una persona lleva a cabo una actividad criminal.
- vict_age: Edad de la víctima
- vict_sex: Sexo de la víctima.
- vict_descent: Código de descendencia de la víctima
- premis_cd: El Código de Instalación es un código de 3 dígitos que identifica el tipo de lugar donde ocurrió el incidente
- premis_desc: Descripción de la Instalación.
- weapon_used_cd: El Código de Arma es un código de 3 dígitos que identifica el tipo de arma utilizada en el incidente
- weapon_desc: Descripción del Arma.
- status: Estado del caso. (IC es el valor predeterminado)
- status_desc: Define el Código de Estado proporcionado.
- crm_cd_1: Indica el crimen cometido. El Código de Crimen 1 es el principal y el más grave. Los Códigos de Crimen 2, 3 y 4 son, respectivamente, delitos menos graves. Los números de clase de crimen más bajos son más graves.
- crm_cd_2: Puede contener un código para un crimen adicional, menos grave que el Código de Crimen 1.
- crm_cd_3: Puede contener un código para un crimen adicional, menos grave que el Código de Crimen 1.
- crm_cd_4: Puede contener un código para un crimen adicional, menos grave que el Código de Crimen 1.
- location: Dirección postal del incidente del crimen redondeada al centenar de la cuadra más cercana para mantener el anonimato.
- cross_street: El nombre de la calle que se cruza con la calle principal donde ocurrió el incidente.
- lat: Latitud.
- lon: Longitud.

Códigos de descendencia: 
- A: Otros Asiáticos
- B: Afroamericano
- C: Chino
- D: Camboyano
- F: Filipino
- G: Guamés
- H: Hispano/Latino/Mexicano
- I: Indio Americano/Nativo de Alaska
- J: Japonés
- K: Coreano
- L: Laosiano
- O: Otro
- P: Isleño del Pacífico
- S: Samoano
- U: Hawaiano
- V: Vietnamita
- W: Blanco
- X: Desconocido
- Z: Hindú Asiático

Revisamos los tipos de datos de cada variable observando entre el .info() y la viasualización de la propia tabla y no se requieren cambios 

También observamos en el count que hay nulos en varias columnas que trataremos posteriormente

In [10]:
pd.set_option('display.max_columns', None)
df.sample()

,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,vict_descent,premis_cd,premis_desc,weapon_used_cd,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
dr_no,,,,,,,,,,,,,,,,,,,,,,,,,,,
231907332,03/23/2023 12:00:00 AM,03/23/2023 12:00:00 AM,735,19,Mission,1998,1,440,THEFT PLAIN - PETTY ($950 & UNDER),0325,30,M,H,243.0,"DIY CENTER (LOWE'S,HOME DEPOT,OSH,CONTRACTORS ...",NaN,NaN,IC,Invest Cont,440.0,NaN,NaN,NaN,7800 VAN NUYS BL,NaN,34.2153,-118.4409


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331647 entries, 201818775 to 240117395
Data columns (total 27 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date_rptd       331647 non-null  object 
 1   date_occ        331647 non-null  object 
 2   time_occ        331647 non-null  int64  
 3   area            331647 non-null  int64  
 4   area_name       331647 non-null  object 
 5   rpt_dist_no     331647 non-null  int64  
 6   part_1_2        331647 non-null  int64  
 7   crm_cd          331647 non-null  int64  
 8   crm_cd_desc     331647 non-null  object 
 9   mocodes         281418 non-null  object 
 10  vict_age        331647 non-null  int64  
 11  vict_sex        283650 non-null  object 
 12  vict_descent    283646 non-null  object 
 13  premis_cd       331642 non-null  float64
 14  premis_desc     331464 non-null  object 
 15  weapon_used_cd  107899 non-null  float64
 16  weapon_desc     107899 non-null  object 
 17  stat

## 4. TRATAMIENTO DE VARIABLES

### FORMATO DE VARIABLES 

Las columnas 'date_rptd' y 'date_occ' vamos a eliminar la hora porque en todos los registros nos dicen que ha sido a las '12:00:00 AM'. Esta información es errónea según la columna 'time_occ' y además no nos aporta información como variable porque es una constante en todos los registros

In [12]:
df.sample()

,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,vict_descent,premis_cd,premis_desc,weapon_used_cd,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
dr_no,,,,,,,,,,,,,,,,,,,,,,,,,,,
241410569,06/16/2024 12:00:00 AM,06/12/2024 12:00:00 AM,1820,14,Pacific,1455,2,354,THEFT OF IDENTITY,0377,28,M,O,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,12400 WALSH AV,NaN,33.9913,-118.4212


Eliminamos la parte de la hora

In [13]:
df['date_rptd'] = df['date_rptd'].str.split(' ').str[0]
df['date_occ'] = df['date_occ'].str.split(' ').str[0]

Hacemos un datetime de las columnas de fechas

In [14]:
df.sample()

,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,vict_descent,premis_cd,premis_desc,weapon_used_cd,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
dr_no,,,,,,,,,,,,,,,,,,,,,,,,,,,
210600633,07/24/2021,07/23/2021,2100,6,Hollywood,636,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),1822 1300 1307 0344,25,F,A,101.0,STREET,NaN,NaN,IC,Invest Cont,420.0,998.0,NaN,NaN,WILCOX,YUCCA,34.1039,-118.331


In [15]:
df['date_rptd'] = pd.to_datetime(df['date_rptd'], format='%m/%d/%Y')
df['date_occ'] = pd.to_datetime(df['date_occ'], format='%m/%d/%Y')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331647 entries, 201818775 to 240117395
Data columns (total 27 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date_rptd       331647 non-null  datetime64[ns]
 1   date_occ        331647 non-null  datetime64[ns]
 2   time_occ        331647 non-null  int64         
 3   area            331647 non-null  int64         
 4   area_name       331647 non-null  object        
 5   rpt_dist_no     331647 non-null  int64         
 6   part_1_2        331647 non-null  int64         
 7   crm_cd          331647 non-null  int64         
 8   crm_cd_desc     331647 non-null  object        
 9   mocodes         281418 non-null  object        
 10  vict_age        331647 non-null  int64         
 11  vict_sex        283650 non-null  object        
 12  vict_descent    283646 non-null  object        
 13  premis_cd       331642 non-null  float64       
 14  premis_desc     331464 non-nul

In [16]:
df.sample()

,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,vict_descent,premis_cd,premis_desc,weapon_used_cd,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
dr_no,,,,,,,,,,,,,,,,,,,,,,,,,,,
210307188,2021-03-08,2021-03-05,2000,3,Southwest,393,1,331,THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND ...,1822 1606 0344 0385 1300 1202,69,M,B,101.0,STREET,NaN,NaN,IC,Invest Cont,331.0,NaN,NaN,NaN,4000 SUTRO AV,NaN,34.0074,-118.329


### DUPLICADOS

Comprobamos si hay duplicados y hacemos comparaciones para asegurarnos

In [17]:
duplicados = df.duplicated().sum()
print(f'Recuento de duplicados en df: ', duplicados)

Recuento de duplicados en df:  412


Nos salen que hay duplicados de las 27 columnas y que solamente es diferente el índice. Hacemos unas visualizaciones para comprobar que son realmente duplicados y vemos que es cierto, por lo que borramos todos los duplicados

In [18]:
lineas_original = len(df)
lineas_original

331647

In [19]:
df.loc[df.duplicated()].head(2)

,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,vict_descent,premis_cd,premis_desc,weapon_used_cd,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
dr_no,,,,,,,,,,,,,,,,,,,,,,,,,,,
201914909,2020-09-24,2020-09-24,304,19,Mission,1935,1,310,BURGLARY,1402 0344,22,X,X,221.0,PUBLIC STORAGE,NaN,NaN,IC,Invest Cont,310.0,998.0,NaN,NaN,15200 BRAND BL,NaN,34.2719,-118.4592
201914906,2020-09-24,2020-09-24,304,19,Mission,1935,1,310,BURGLARY,1402 0344,22,X,X,221.0,PUBLIC STORAGE,NaN,NaN,IC,Invest Cont,310.0,998.0,NaN,NaN,15200 BRAND BL,NaN,34.2719,-118.4592


In [20]:
# Buscamos una en concreto por hacer la comprobación
df.loc[(df.mocodes == '0421 1528 1822 0903 2053')]

,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,vict_descent,premis_cd,premis_desc,weapon_used_cd,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
dr_no,,,,,,,,,,,,,,,,,,,,,,,,,,,
220706172,2022-02-26,2022-02-26,1200,7,Wilshire,752,2,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,0421 1528 1822 0903 2053,0,M,O,501.0,SINGLE FAMILY DWELLING,511.0,VERBAL THREAT,IC,Invest Cont,930.0,946.0,NaN,NaN,1100 S ORLANDO AV,NaN,34.0572,-118.3739
220706177,2022-02-26,2022-02-26,1200,7,Wilshire,752,2,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,0421 1528 1822 0903 2053,0,F,O,501.0,SINGLE FAMILY DWELLING,511.0,VERBAL THREAT,IC,Invest Cont,930.0,946.0,NaN,NaN,1100 S ORLANDO AV,NaN,34.0572,-118.3739
220706174,2022-02-26,2022-02-26,1200,7,Wilshire,752,2,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,0421 1528 1822 0903 2053,0,F,O,501.0,SINGLE FAMILY DWELLING,511.0,VERBAL THREAT,IC,Invest Cont,930.0,946.0,NaN,NaN,1100 S ORLANDO AV,NaN,34.0572,-118.3739
220706169,2022-02-26,2022-02-26,1200,7,Wilshire,752,2,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,0421 1528 1822 0903 2053,0,M,O,501.0,SINGLE FAMILY DWELLING,511.0,VERBAL THREAT,IC,Invest Cont,930.0,946.0,NaN,NaN,1100 S ORLANDO AV,NaN,34.0572,-118.3739


Eliminamos los duplicados

In [21]:
df.drop_duplicates(inplace=True)

In [22]:
lineas_despues = len(df)

diff = lineas_original - lineas_despues

print("Filas antes:", lineas_original)
print("Filas después:", lineas_despues)
print("Diferencia:", diff)

Filas antes: 331647
Filas después: 331235
Diferencia: 412


### VALORES ÚNICOS

Comprobamos los valores únicos en la tabla para desestimar variables. En este caso, no hay constantes

In [23]:
df.nunique().sort_values(ascending=True)

# Si hubiera alguna columna que eliminar haríamos df.drop(columns = '...', inplace=True)

part_1_2               2
vict_sex               4
crm_cd_4               4
status                 6
status_desc            6
vict_descent          20
area_name             21
area                  21
crm_cd_3              24
weapon_used_cd        78
weapon_desc           78
crm_cd_2              96
vict_age             103
crm_cd               139
crm_cd_desc          139
crm_cd_1             141
premis_desc          301
premis_cd            308
rpt_dist_no         1167
time_occ            1439
date_occ            1854
date_rptd           1868
lon                 4885
lat                 5176
cross_street        6540
location           50628
mocodes           117230
dtype: int64

### VALORES NULOS

Comprobamos los nulos que existen según el tipo de columna: Categórica y numérica

Primero haremos las comprobaciones, después expondré las conclusiones y a continuación realizaré los cambios

In [24]:
df.select_dtypes(include='number').isna().sum().sort_values(ascending=False)

crm_cd_4          331210
crm_cd_3          330509
crm_cd_2          308470
weapon_used_cd    223477
premis_cd              5
crm_cd_1               4
vict_age               0
crm_cd                 0
part_1_2               0
rpt_dist_no            0
area                   0
time_occ               0
lat                    0
lon                    0
dtype: int64

In [25]:
df.select_dtypes(exclude='number').isna().sum().sort_values(ascending=False)

cross_street    280321
weapon_desc     223477
mocodes          50061
vict_descent     47839
vict_sex         47835
premis_desc        183
status               1
date_occ             0
date_rptd            0
crm_cd_desc          0
area_name            0
status_desc          0
location             0
dtype: int64

In [26]:
df.vict_descent.unique()

array(['B', 'A', 'H', 'X', 'W', 'O', nan, 'K', 'C', 'J', 'I', 'P', 'V',
       'F', 'Z', 'U', 'L', 'S', 'G', 'D', '-'], dtype=object)

In [27]:
df.loc[df.vict_descent=='-']

,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,vict_descent,premis_cd,premis_desc,weapon_used_cd,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
dr_no,,,,,,,,,,,,,,,,,,,,,,,,,,,
230908075,2023-04-12,2023-03-30,1200,9,Van Nuys,952,2,649,DOCUMENT FORGERY / STOLEN FELONY,1822 0924 1501,38,M,-,203.0,OTHER BUSINESS,NaN,NaN,IC,Invest Cont,649.0,NaN,NaN,NaN,15300 MAGNOLIA BL,NaN,34.1684,-118.4586


In [28]:
df.vict_sex.unique()

array(['F', 'M', 'X', nan, 'H'], dtype=object)

In [29]:
df.loc[df.vict_sex=='-']

,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,vict_descent,premis_cd,premis_desc,weapon_used_cd,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
dr_no,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [30]:
df.loc[(df.premis_cd.notna())&(df.premis_desc.isna())].premis_cd.value_counts()

premis_cd
418.0    123
256.0     44
974.0      4
972.0      4
973.0      2
976.0      1
Name: count, dtype: int64

In [31]:
df.loc[(df.crm_cd_1.isna())]

,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,vict_descent,premis_cd,premis_desc,weapon_used_cd,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
dr_no,,,,,,,,,,,,,,,,,,,,,,,,,,,
210118616,2021-10-13,2021-10-13,1840,1,Central,162,1,442,SHOPLIFTING - PETTY THEFT ($950 & UNDER),2004 0325,0,X,X,404.0,DEPARTMENT STORE,NaN,NaN,IC,Invest Cont,NaN,442.0,NaN,NaN,700 W 7TH ST,NaN,34.0480,-118.2577
220805565,2022-02-09,2022-02-09,245,8,West LA,842,2,888,TRESPASSING,1501,21,F,H,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,NaN,888.0,NaN,NaN,11700 WILSHIRE BL,NaN,34.0495,-118.4609
230114704,2023-06-22,2023-06-22,1801,1,Central,162,1,442,SHOPLIFTING - PETTY THEFT ($950 & UNDER),0325,25,M,H,404.0,DEPARTMENT STORE,NaN,NaN,IC,Invest Cont,NaN,442.0,NaN,NaN,700 S FLOWER ST,NaN,34.0487,-118.2588
230106125,2023-02-01,2023-02-01,1855,1,Central,129,1,330,BURGLARY FROM VEHICLE,0344 1609,32,M,B,101.0,STREET,NaN,NaN,IC,Invest Cont,NaN,330.0,NaN,NaN,GAREY,JACKSON,34.0513,-118.2344


In [32]:
df.loc[(df.status.isna())]

,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,vict_descent,premis_cd,premis_desc,weapon_used_cd,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
dr_no,,,,,,,,,,,,,,,,,,,,,,,,,,,
241810568,2024-06-08,2024-05-24,1900,18,Southeast,1871,1,510,VEHICLE - STOLEN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNK,510.0,NaN,NaN,NaN,13500 S FIGUEROA ST,NaN,33.9092,-118.2827


In [33]:
df.loc[(df.crm_cd_1 == 510.0)]

,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,vict_descent,premis_cd,premis_desc,weapon_used_cd,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
dr_no,,,,,,,,,,,,,,,,,,,,,,,,,,,
201712347,2020-08-20,2020-07-31,1400,17,Devonshire,1735,1,510,VEHICLE - STOLEN,NaN,0,NaN,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,17800 LOS ALIMOS ST,NaN,34.2661,-118.5234
201818504,2020-10-08,2020-10-08,200,18,Southeast,1841,1,510,VEHICLE - STOLEN,NaN,0,NaN,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,10800 S VERMONT AV,NaN,33.9378,-118.2916
211604055,2021-01-03,2020-12-31,2100,16,Foothill,1689,1,510,VEHICLE - STOLEN,NaN,0,NaN,NaN,108.0,PARKING LOT,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,CLYBOURN AV,SAN FERNANDO RD,34.2226,-118.3713
201812802,2020-06-19,2020-06-18,1900,18,Southeast,1832,1,510,VEHICLE - STOLEN,NaN,0,NaN,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,10700 S BROADWAY,NaN,33.9392,-118.2781
201505788,2020-02-06,2020-02-05,1630,15,N Hollywood,1523,1,510,VEHICLE - STOLEN,NaN,0,NaN,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,6300 COLDWATER CANYON AV,NaN,34.1849,-118.4137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241909262,2024-06-02,2024-06-01,1800,19,Mission,1935,1,510,VEHICLE - STOLEN,NaN,0,NaN,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,14700 MACLAY ST,NaN,34.2745,-118.4529
242104707,2024-01-18,2024-01-18,200,21,Topanga,2144,1,510,VEHICLE - STOLEN,NaN,0,NaN,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,22500 WELBY WY,NaN,34.1928,-118.6174
241905147,2024-01-30,2024-01-15,2200,19,Mission,1981,1,510,VEHICLE - STOLEN,NaN,0,NaN,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,8600 N SEPULVEDA BL,NaN,34.2267,-118.4704
